In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import wandb
import xgboost as xgb

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import wandb
import xgboost as xgb

# Active Learning 반복 과정
n_queries = 20
batch_size = 1750  # 각 반복에서 선택할 배치 크기

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="New_drug",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": n_queries,
    }
)



# CSV 파일 경로
# labeled_data_csv = "./origin_data/train.csv"  # 레이블이 있는 데이터 파일
# unlabeled_data_csv = "./data/unlabeld_data2.csv"  # 레이블이 없는 데이터 파일
labeled_data_csv = "./data/finger_maccs_fp_train.csv"  # 레이블이 있는 데이터 파일
unlabeled_data_csv = "./data/unlabeled_maccs.csv"  # 레이블이 없는 데이터 파일

# CSV 파일 읽기
labeled_data = pd.read_csv(labeled_data_csv).drop(columns=["id", "SMILES"])
unlabeled_data = pd.read_csv(unlabeled_data_csv).drop(columns=["SMILES"])

labeled_data = labeled_data.fillna(0)
unlabeled_data = unlabeled_data.fillna(0)


# 레이블이 있는 데이터 추출
X_labeled = labeled_data.drop(columns=['MLM', 'HLM'])
Y_labeled = labeled_data[['MLM', 'HLM']]

###
# # 초기 학습 데이터 선택 (랜덤으로 선택)
# initial_idx = np.random.choice(range(len(X_labeled)), size=1000, replace=False)
# X_initial = X_labeled.iloc[initial_idx]
# Y_initial = Y_labeled.iloc[initial_idx]

# 초기 학습 데이터 선택 (80%)
test_size = 0.2  # 전체 데이터 중 테스트 데이터 비율 (예: 20%)
X_initial, X_test, Y_initial, Y_test = train_test_split(X_labeled, Y_labeled, test_size=test_size, random_state=42)
X_test1, X_test2, Y_test1, Y_test2 = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)
###

# 초기 모델 학습 (랜덤 포레스트 회귀 모델 사용)
initial_model = RandomForestRegressor()
initial_model.fit(X_initial, Y_initial)

for i in tqdm(range(n_queries)):
    # 예측 오차를 계산하여 불확실성을 측정
    Y_pred = initial_model.predict(unlabeled_data)  # 레이블이 없는 데이터에 대한 예측
    #uncertainty = np.abs(Y_pred - np.mean(Y_pred, axis=0))
    uncertainty = 1 - np.max(Y_pred, axis=1)

    # 불확실성이 가장 큰 상위 배치 크기만큼 데이터 포인트 선택
    #query_indices = np.argsort(uncertainty.sum(axis=1))[-batch_size:]
    query_indices = np.argsort(uncertainty)[-batch_size:]

    for query_idx in query_indices:
        # 쿼리된 데이터는 학습 데이터에 추가
        X_query = unlabeled_data.iloc[query_idx:query_idx+1]
        Y_query = Y_pred[query_idx:query_idx+1]  # 모델 예측값을 사용

        X_initial = pd.concat([X_initial, X_query])
        Y_initial = np.vstack([Y_initial, Y_query])

    # 쿼리된 데이터는 레이블이 없는 데이터에서 제거 (선택된 데이터만큼 제거)
    unlabeled_data = unlabeled_data.drop(unlabeled_data.index[query_indices])

    # 새로운 모델을 학습하여 계속 진행
    initial_model = RandomForestRegressor()
    initial_model.fit(X_initial, Y_initial)

    Y_pred = initial_model.predict(X_test1)

    rmse_MLM = np.sqrt(mean_squared_error(Y_test1["MLM"], Y_pred[:,0]))
    rmse_HLM = np.sqrt(mean_squared_error(Y_test1["HLM"], Y_pred[:,1]))
    loss = (rmse_HLM+rmse_MLM)/2
        # log metrics to wandb
    wandb.log({"loss": loss})


Y_pred = initial_model.predict(X_test2)

rmse_MLM = np.sqrt(mean_squared_error(Y_test2["MLM"], Y_pred[:,0]))
rmse_HLM = np.sqrt(mean_squared_error(Y_test2["HLM"], Y_pred[:,1]))
rmse = (rmse_HLM+rmse_MLM)/2
print("RMSE:", rmse)


# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: pook0612 (limbw). Use `wandb login --relogin` to force relogin


100%|██████████| 20/20 [49:53<00:00, 149.66s/it]

RMSE: 33.96153455316527


loss,▃▆▄▅██▄▃▁▄▆▄▄▃▃▄▅▂▃▅
loss,32.22674


In [2]:
import joblib

# 모델 저장
joblib.dump(initial_model, './AutogluonModels/initial_model_maccs_SS.pkl')


['./AutogluonModels/initial_model_maccs_SS.pkl']

In [18]:
# import joblib
# # 모델 로드
# loaded_model = joblib.load('initial_model1.pkl')

# # 로드한 모델로 예측 등을 수행할 수 있음
# loaded_predictions = loaded_model.predict(X_test)


In [15]:
# Y_pred_df = pd.DataFrame(Y_pred)
# Y_pred_df.to_csv("./data/SS_valid_pred.csv")

In [3]:
#test_csv = "./origin_data/test.csv"
test_csv = "./data/finger_maccs_fp_test.csv"
test_data = pd.read_csv(test_csv).drop(columns=["id", "SMILES"])
test_data = test_data.fillna(0)

Y_test_pred = initial_model.predict(test_data)
# predict
df_submission = pd.read_csv("./origin_data/sample_submission.csv")
df_submission["MLM"] = Y_test_pred[:,0]
df_submission["HLM"] = Y_test_pred[:,1]
df_submission.to_csv("./submission/submission_SSwandb_maccs_LBW.csv", index = False, encoding = "utf-8-sig")

In [2]:
autog = pd.read_csv("./submission/autog.csv")
ss2 = pd.read_csv("./submission/submission_SS2_LBW.csv")
ss = pd.read_csv("./submission/submission_SS_LBW.csv")
ss35 = pd.read_csv("./submission/submission_SS35k_LBW.csv")